In [1]:
import pymeshlab
import glob
import os
import open3d as o3
import time
import numpy as np
import copy
import trimesh
import scipy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def cleaning_repairing(ms):
    ms.meshing_merge_close_vertices()
    # print('meshing_merge_close_vertices: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_merge_close_vertices: #face_mesh=',ms.current_mesh().face_number())

    #Merge together per-wedge texture coords that are very close. Used to correct apparent texture seams that can arise from numerical approximations
    ms.apply_texcoord_merge_per_wedge()
    # print('apply_texcoord_merge_per_wedge: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('apply_texcoord_merge_per_wedge: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_duplicate_faces()
    # print('meshing_remove_duplicate_faces: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_duplicate_faces: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_duplicate_vertices()
    # print('meshing_remove_duplicate_vertices: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_duplicate_vertices: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_folded_faces()
    # print('meshing_remove_folded_faces: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_folded_faces: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_null_faces()
    # print('meshing_remove_null_faces: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_null_faces: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_connected_component_by_diameter()
    # print('meshing_remove_connected_component_by_diameter: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_connected_component_by_diameter: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_connected_component_by_face_number()
    # print('meshing_remove_connected_component_by_face_number: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_connected_component_by_face_number: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_t_vertices()
    # print('meshing_remove_t_vertices: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_t_vertices: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_remove_unreferenced_vertices()
    # print('meshing_remove_unreferenced_vertices: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_remove_unreferenced_vertices: #face_mesh=',ms.current_mesh().face_number())

    # Split non Manifold vertices until it becomes 2-Manifold.
    ms.meshing_repair_non_manifold_edges()
    # print('meshing_repair_non_manifold_edges: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_repair_non_manifold_edges: #face_mesh=',ms.current_mesh().face_number())

    ms.meshing_repair_non_manifold_vertices()
    # print('meshing_repair_non_manifold_vertices: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_repair_non_manifold_vertices: #face_mesh=',ms.current_mesh().face_number())

    # Try to snap together adjacent borders that are slightly mismatched.
    # This situation can happen on badly triangulated adjacent patches defined by high order surfaces
    # ms.meshing_snap_mismatched_borders()
    # print('meshing_snap_mismatched_borders: #vertex_mesh=',ms.current_mesh().vertex_number())
    # print('meshing_snap_mismatched_borders: #face_mesh=',ms.current_mesh().face_number())

    return ms
def filtering(ms,first_num_samp,sec_num_samp,trd_num_samp):
    ms=cleaning_repairing(ms)
    ms.generate_sampling_montecarlo(samplenum=first_num_samp)
    ms.generate_surface_reconstruction_screened_poisson()
    ms.generate_sampling_poisson_disk (samplenum =sec_num_samp)
    ms.generate_surface_reconstruction_screened_poisson()
    print('sampling_poisson_disk - screened_poisson: #vertx_mesh=',ms.current_mesh().vertex_number())
    print('sampling_poisson_disk - screened_poisson: #face_mesh=',ms.current_mesh().face_number())
    ms=cleaning_repairing(ms)
    ms.generate_sampling_montecarlo(samplenum=trd_num_samp)
    ms.generate_surface_reconstruction_screened_poisson()
    ms=cleaning_repairing(ms)
    ms.generate_sampling_montecarlo(samplenum=200000)
    ms.generate_surface_reconstruction_screened_poisson()
    ms=cleaning_repairing(ms)
    print('montecarlo sampling - screened_poisson: #vertx_mesh=',ms.current_mesh().vertex_number())
    print('montecarlo sampling - screened_poisson: #face_mesh=',ms.current_mesh().face_number())
# smoothing
    for tt in range(2):
        ms.apply_coord_laplacian_smoothing_surface_preserving(angledeg =100, iterations =100)
    for tt in range(8):
        ms.apply_coord_unsharp_mask(weight =0.1,weightorig =1,iterations=500) #MeshLab filter name: ‘UnSharp Mask Geometry’
    ms=cleaning_repairing(ms)
    ms.save_current_mesh(path_save+'\\'+'meshlab_'+os.path.basename(address_stls[stl_num]))

In [3]:
cartilage_type='FC', #TC or FC
str_stls=0 # first sample number
end_stls=9 #last sample number

address_stls=glob.glob('D:FC\*.stl') # all original data
address_save='D:\FC'# for saving
address_stls,len(address_stls)
path_save=address_save+'\\'+'filtered'



In [4]:
# mesh repairing, sampling, and smoothing
for stl_num in range (str_stls,end_stls):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(address_stls[stl_num])

    if cartilage_type=='TC':
        filtering(ms,7000,7000,5000)
    else:
        filtering(ms,20000,20000,15000)

In [23]:
# Rigid registration
def refresh_mesh(mesh):
    mesh.remove_duplicated_vertices()
    mesh.remove_duplicated_triangles()
    mesh.remove_degenerate_triangles()
    mesh.remove_unreferenced_vertices()
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
def distance_pp(pcd1, pcd2):
    return np.mean(pcd1.compute_point_cloud_distance(pcd2))

gray         = [0.5, 0.5, 0.5]
def x_to_pcd(x, color=gray):
    pcd = o3.geometry.PointCloud()
    pcd.points = o3.utility.Vector3dVector(x)
    pcd.estimate_normals()
    pcd.paint_uniform_color(color)
    return pcd
def compute_sigma(source, target):
    source_pcd = x_to_pcd(source)
    target_pcd = x_to_pcd(target)
    dis = np.mean([distance_pp(source_pcd, target_pcd), distance_pp(target_pcd, source_pcd)])
    sigma2 = dis * 0.5
    return sigma2
def read_mesh(filename):
    mesh = o3.io.read_triangle_mesh(filename)
    refresh_mesh(mesh)
    return mesh
def write_mesh(name, mesh):
    time.sleep(1)
    o3.io.write_triangle_mesh(name, mesh)
def cpd_rigid(Y, X):
    w             = 0
    max_iteration = 50
    scale         = True
    tolerance     = 1.0e-3

    sigma2 = compute_sigma(Y, X)
    N, D = X.shape
    M = Y.shape[0]
    iteration = 0
    TY = Y

    Sigma2 = np.empty(0)
    while iteration <= max_iteration:
        # SAVE
        Sigma2 = np.append(Sigma2, sigma2)

        # EXPECTATION
        P = np.sum((X[None, :, :] - TY[:, None, :]) ** 2, axis=2)
        P = np.exp(-P / (2 * sigma2))
        c = (2 * np.pi * sigma2) ** (D / 2) * w / (1. - w) * M / N
        den = np.sum(P, axis=0, keepdims=True)
        den = np.clip(den, np.finfo(X.dtype).eps, None) + c
        P = np.divide(P, den)
        Pt1 = np.sum(P, axis=0)
        P1 = np.sum(P, axis=1)
        Np = np.sum(P1)
        PX = np.matmul(P, X)

        # MAXIMIZATION
        # UPDATE TRANSFORM
        muX = np.divide(np.sum(PX, axis=0), Np)
        muY = np.divide(np.sum(np.dot(np.transpose(P), Y), axis=0), Np)
        X_hat = X - np.tile(muX, (N, 1))
        Y_hat = Y - np.tile(muY, (M, 1))
        YPY = np.dot(np.transpose(P1), np.sum(np.multiply(Y_hat, Y_hat), axis=1))
        A = np.dot(np.transpose(X_hat), np.transpose(P))
        A = np.dot(A, Y_hat)
        U, _, V = np.linalg.svd(A, full_matrices=True)
        C = np.ones((D,))
        C[D - 1] = np.linalg.det(np.dot(U, V))
        R = np.transpose(np.dot(np.dot(U, np.diag(C)), V))
        if scale is True:
            s = np.trace(np.dot(np.transpose(A), np.transpose(R))) / YPY
        else:
            s = 1
        t = np.transpose(muX) - s * np.dot(np.transpose(R), np.transpose(muY))
        # TRANSFORM SHAPES
        TY = s * np.dot(Y, R) + t
        # UPDATE VARIANCE
        trAR = np.trace(np.dot(A, R))
        xPx = np.dot(np.transpose(Pt1), np.sum(np.multiply(X_hat, X_hat), axis=1))
        sigma2 = (xPx - s * trAR) / (Np * D)
        if sigma2 <= 0:
            sigma2 = tolerance

        # ITERATION
        iteration += 1

    return TY, R, s, t
def rigid_transform(mesh, R, s, t):
    Mesh = copy.deepcopy(mesh)
    x = np.array(Mesh.vertices)
    x = s * np.dot(x, R) + t
    Mesh.vertices = o3.utility.Vector3dVector(x)
    refresh_mesh(Mesh)
    return Mesh

for stl_num in range (str_stls,end_stls):
    trg_addss=glob.glob('D:FC\Swin\*.stl')
    src_addss=glob.glob('D:FC\filtered\*.stl')                  
    target_mesh = read_mesh(trg_addss[stl_num])
    source_mesh= read_mesh(src_addss[stl_num])
    for j in range(5):
        source_a = source_mesh.get_surface_area()
        target_a = target_mesh.get_surface_area()
        source_n=2000
        target_n = np.int_(source_n / source_a * target_a)
        source_pcd = source_mesh.sample_points_poisson_disk(number_of_points=source_n)
        target_pcd = target_mesh.sample_points_poisson_disk(number_of_points=target_n)
        source = np.array(source_pcd.points)
        target = np.array(target_pcd.points)
        source, R, s, t = cpd_rigid( source, target)
        source_mesh = rigid_transform(source_mesh, R, s, t)
    write_mesh(path_save+'\\registered\\meshlab_'+os.path.basename(address_stls[stl_num]), source_mesh)